In [9]:
# Importar librerías necesarias
from openai import OpenAI
import pandas as pd
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import random

# Crear una instancia del cliente de OpenAI
client = OpenAI()

In [10]:
# Dataset de pruebas
# Establecer semilla para reproducibilidad
random.seed(42)
np.random.seed(42)

# Crear una base de datos ficticia: "Estudio de comportamiento de aves en zonas urbanas y rurales"
n = 100

df_main = pd.DataFrame({
    "ID_observación": [f"OBS_{i+1:04d}" for i in range(n)],
    "Especie": np.random.choice(
        ["Zorzal", "Chincol", "Tenca", "Picaflor", "Gorrión", "Peuco", "Queltehue", "Diucón"], size=n),
    "Zona": np.random.choice(["Urbana", "Rural", "Suburbana"], size=n),
    "Hora_observación": np.random.choice(
        ["Mañana", "Mediodía", "Tarde", "Noche"], size=n),
    "Duración_observación_min": np.random.normal(loc=30, scale=12, size=n).round(1),
    "Cantidad_individuos": np.random.poisson(lam=4, size=n),
    "Temperatura_C": np.random.normal(loc=19, scale=4.5, size=n).round(1),
    "Humedad_%": np.random.normal(loc=55, scale=15, size=n).round(1),
    "Viento_kmh": np.random.normal(loc=12, scale=5, size=n).round(1),
    "Comportamiento": np.random.choice(
        ["Alimentación", "Canto", "Vuelo", "Reposo", "Interacción", "Defensa de territorio"], size=n),
    "Interacción_humana": np.random.choice(["Sí", "No"], size=n, p=[0.4, 0.6]),
    "Presencia_predador": np.random.choice(["Sí", "No"], size=n, p=[0.3, 0.7]),
    "Fecha_observación": pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2023-12-31"), size=n)),
    "Observador": np.random.choice(
        ["Luis", "Camila", "Ignacio", "Valentina", "Marcelo", "Antonia"], size=n),
})

df_sample = df_main.head(3).to_string(index=False)

In [11]:
# Desplegar el DataFrame

df_main.head(10).style.set_table_styles([{
    'selector': 'th',  # Estilo para las celdas de la cabecera
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}, {
    'selector': 'td',  # Estilo para las celdas de datos
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}])

,ID_observación,Especie,Zona,Hora_observación,Duración_observación_min,Cantidad_individuos,Temperatura_C,Humedad_%,Viento_kmh,Comportamiento,Interacción_humana,Presencia_predador,Fecha_observación,Observador
0,OBS_0001,Queltehue,Suburbana,Noche,30.300000,1,24.400000,35.000000,6.500000,Reposo,No,No,2023-06-18 00:00:00,Marcelo
1,OBS_0002,Picaflor,Rural,Noche,15.600000,5,22.000000,53.200000,0.100000,Interacción,No,Sí,2023-03-30 00:00:00,Luis
2,OBS_0003,Gorrión,Rural,Noche,39.500000,4,16.200000,89.200000,9.800000,Vuelo,Sí,Sí,2023-06-25 00:00:00,Camila
3,OBS_0004,Queltehue,Rural,Noche,35.200000,3,18.200000,13.200000,6.100000,Alimentación,No,No,2023-09-26 00:00:00,Marcelo
4,OBS_0005,Tenca,Rural,Noche,16.800000,4,17.500000,68.000000,25.000000,Defensa de territorio,No,Sí,2023-03-14 00:00:00,Camila
5,OBS_0006,Diucón,Rural,Mediodía,21.800000,3,16.900000,70.900000,8.700000,Vuelo,No,No,2023-09-02 00:00:00,Valentina
6,OBS_0007,Gorrión,Rural,Mediodía,40.700000,3,17.200000,34.600000,8.600000,Vuelo,Sí,No,2023-03-18 00:00:00,Valentina
7,OBS_0008,Gorrión,Suburbana,Tarde,37.300000,5,17.100000,48.000000,4.400000,Canto,No,Sí,2023-06-08 00:00:00,Valentina
8,OBS_0009,Queltehue,Suburbana,Noche,21.400000,3,16.900000,40.000000,10.700000,Canto,Sí,No,2023-11-20 00:00:00,Antonia
9,OBS_0010,Chincol,Rural,Mediodía,26.800000,2,23.300000,82.800000,8.700000,Canto,Sí,No,2023-04-08 00:00:00,Antonia


In [12]:
# Configuración GPT
gpt_model = 'gpt-4o' # gpt-3.5-turbo-0125 o gpt-4o

parametros = {
    "max_tokens": 1000,
    "temperature": 0.5,
    "top_p": 0.8,
    "presence_penalty": 0.3,
    "frequency_penalty": 0.5
}

In [13]:
# Crear el Contexto del Sistema

# Formato de respuesta y ejecución
contexto_formato = (
    "Tu salida debe estar en formato JSON, con una única clave 'code'.\n"
    "El valor debe ser una cadena de texto que contenga código Python ejecutable.\n"
    "Usa `df_main` como referencia al DataFrame completo disponible en la variable global.\n"
    "Solo se te muestra un resumen visual (df_sample).\n"
    "El código debe estar listo para ser ejecutado con `exec()`, sin celdas mágicas ni prints redundantes.\n"
    "Importa explícitamente cualquier librería que vayas a utilizar.\n"
    "Inicializa gráficos con `sns.set_theme()` si incluyes visualizaciones.\n"
    "No generes o modifiques DataFrames externos salvo que se indique expresamente en la pregunta.\n"
)

# Enfoque temático y rol del asistente
contexto_tema = (
    "Eres un asistente experto en análisis de datos con Python.\n"
    "Tu tarea es ayudar al usuario generando código que analice relaciones entre variables\n"
    "Puedes usar técnicas de agrupación, filtrado, visualización o resumen estadístico.\n"
    "Responde siempre con código funcional y enfocado al análisis solicitado.\n"
)

# Contexto completo combinado
contexto_sistema = contexto_formato + "\n" + contexto_tema

In [14]:
# Crear el Prompt e incorpora la Pregunta
def obtener_respuesta_con_dataframe(pregunta):
    prompt = (
        f"Se muestra un muestra del DataFrame como referencia visual únicamente:\n\n"
        f"df_sample_1:\n{df_sample}\n\n"
        "Estas muestras son ilustrativas. El DataFrame completo está disponible como variable global `df_main`.\n\n"
        f"Pregunta del usuario:\n{pregunta}\n\n"
        "Tu tarea es generar **solo código Python** para responder esta pregunta, en un objeto JSON con la estructura exacta:\n\n"
        "`{\"code\": \"<código Python aquí>\"}`\n\n"
        "Instrucciones clave:\n"
        "- El código debe ser completamente **ejecutable** y no debe incluir comentarios fuera del bloque JSON.\n"
        "- Debes usar las variables `df_main` como referencia al DataFrame completo.\n"
        "- No generes DataFrames desde cero ni cargues archivos a menos que la pregunta lo solicite explícitamente.\n"
        "- Si usas visualizaciones, aplica `sns.set_theme()` y utiliza gráficos de `seaborn` o `matplotlib`.\n"
        "- Usa importaciones estándar como `import pandas as pd`, `import seaborn as sns`, etc., si no están explícitamente presentes.\n\n"
        "No incluyas ninguna explicación textual. Solo retorna el JSON con el bloque de código."
)

    # Consulta API
    try:
        response = client.chat.completions.create(
            model=gpt_model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": contexto_sistema},
                {"role": "user", "content": prompt}
            ],
            **parametros  # Usar el diccionario como argumento con **
        )
        
        respuesta_json = response.choices[0].message.content
        codigo_python_dic = json.loads(respuesta_json)

        # Verificar y asegurar el uso de 'df_main' en el código
        codigo_python = codigo_python_dic['code']
        if 'df' not in codigo_python:
            raise ValueError("El código generado no usa 'df'.")
        return codigo_python
    except Exception as e:
        print("Error al obtener respuesta de OpenAI:", str(e))
        return None

# Crear widgets interactivos
def crear_widgets(codigo_python):
    widget_codigo = widgets.Textarea(
        value=codigo_python,
        layout=widgets.Layout(width='99%', height='300px'),
        style={'font_family': 'monospace'}
    )

    boton_ejecutar = widgets.Button(description="Ejecutar")
    output_resultado = widgets.Output(
        layout=widgets.Layout(width='99%')
    )

    def ejecutar_codigo(b):
        output_resultado.clear_output()
        codigo_editado = widget_codigo.value
        with output_resultado:
            clear_output(wait=True)
            try:
                exec(codigo_editado, globals())
            except Exception as e:
                print("Error al ejecutar el código:", e)

    boton_ejecutar.on_click(ejecutar_codigo)

    return widget_codigo, boton_ejecutar, output_resultado

In [15]:
# Modulo de consulta y respuesta
pregunta = (
    "Puedes mostrar en un histograma de las temperaturas? "
)

codigo_python = obtener_respuesta_con_dataframe(pregunta)
if codigo_python:
    widget_codigo, boton_ejecutar, output_resultado = crear_widgets(codigo_python)
    display(widget_codigo, boton_ejecutar, output_resultado)
else:
    print("No se pudo obtener una respuesta de la API de OpenAI.")

Textarea(value="import seaborn as sns\nimport matplotlib.pyplot as plt\n\nsns.set_theme()\nplt.figure(figsize=…

Button(description='Ejecutar', style=ButtonStyle())

Output(layout=Layout(width='99%'))